In [ ]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

El csv donde tenemos la información censal de cada Comunidad Autónoma tiene algunas columnas que no son redundantes para nuestro análisis, en este csv tendremos que:

Eliminar la columna de Comunidades_y_Ciudades_Autónomas

En el csv con la información de generación de energías renovables y no renovables a nivel nacional deberéis:
Para las columnas value y percentage, redondear los decimales de los valores a 2.
La columna datetime es de tipo string, pero esto a nosotras no nos interesa. Además, en esta columna tiene tanto la fecha como la hora, pero en realidad la hora no nos esta aportando nada de información, es por esto que:
Deberéis crear una columna nueva donde solo tengamos la información de la fecha en formato AÑO / MES / DIA.
Una vez creada la columna anterior, convertirla a datetime.

En el csv con la información de generación de energías renovables y no renovables a nivel de comunidad autónoma deberéis aplicar los mismos cambios que aplicamos en el ejercicio anterior.



Guardar los resultados obtenidos en pickle que usaremos en próximos ejercicios de pair programming. Cada fichero debe ir en un pickle